## Final Assignment Overview: Working with Patient Records and Encounter Notes

In this final assignment, we’ll focus on patient records related to COVID-19 encounters. Our task is to analyze, process, and transform the data while applying the concepts we’ve covered throughout this course. Here's a detailed breakdown of the assignment:

What Are Encounter Notes?
An encounter note is a record that captures details about a patient’s visit with a doctor. It includes both structured and semi-structured information that is crucial for understanding the context of the visit. Here’s what an encounter note typically looks like:

```
AMBULATORY ENCOUNTER NOTE
Date of Service: March 2, 2020 15:45-16:30

DEMOGRAPHICS:
Name: Jeffrey Greenfelder
DOB: 1/16/2005
Gender: Male
Address: 428 Wiza Glen Unit 91, Springfield, Massachusetts 01104
Insurance: Guardian
MRN: 055ae6fc-7e18-4a39-8058-64082ca6d515

PERTINENT MEDICAL HISTORY:
- Obesity 

Recent Visit: Well child visit (2/23/2020)
Immunizations: Influenza vaccine (2/23/2020)

Recent Baseline (2/23/2020):
Height: 155.0 cm
Weight: 81.2 kg
BMI: 33.8 kg/m² (99.1th percentile)
BP: 123/80 mmHg
HR: 92/min
RR: 13/min

SUBJECTIVE:
Adolescent patient presents with multiple symptoms including:
- Cough
- Sore throat
- Severe fatigue
- Muscle pain
- Joint pain
- Fever
Never smoker. Symptoms began recently.

OBJECTIVE:
Vitals:
Temperature: 39.3°C (102.7°F)
Heart Rate: 131.1/min
Blood Pressure: 120/73 mmHg
Respiratory Rate: 27.6/min
O2 Saturation: 75.8% on room air
Weight: 81.2 kg

Laboratory/Testing:
Comprehensive Respiratory Panel:
- Influenza A RNA: Negative
- Influenza B RNA: Negative
- RSV RNA: Negative
- Parainfluenza virus 1,2,3 RNA: Negative
- Rhinovirus RNA: Negative
- Human metapneumovirus RNA: Negative
- Adenovirus DNA: Negative
- SARS-CoV-2 RNA: Positive

ASSESSMENT:
1. Suspected COVID-19 with severe symptoms
2. Severe hypoxemia requiring immediate intervention
3. Tachycardia (HR 131)
4. High-grade fever
5. Risk factors:
   - Obesity (BMI 33.8)
   - Adolescent age

PLAN:
1. Face mask provided for immediate oxygen support
2. Infectious disease care plan initiated
3. Close monitoring required due to:
   - Severe hypoxemia
   - Tachycardia
   - Age and obesity risk factors
4. Parent/patient education on:
   - Home isolation protocols
   - Warning signs requiring emergency care
   - Return precautions
5. Follow-up plan:
   - Daily monitoring during acute phase
   - Virtual check-ins as needed

Encounter Duration: 45 minutes
Encounter Type: Ambulatory
Provider: ID# e2c226c2-3e1e-3d0b-b997-ce9544c10528
Facility: 5103c940-0c08-392f-95cd-446e0cea042a
```


The enocuter contains

* General encounter information: 

  * When the encounter took place: Date and time of the visit.
  * Demographics: Patient’s age, gender, and unique medical record identifier.
  * Encounter details: The reason for the visit, diagnosis, and any associated costs.


* Semi-Structured Notes:

These notes mirror how doctors organize their thoughts and observations during an encounter. They generally follow a SOAP format:

* Subjective: The patient’s subjective description of their symptoms, feelings, and medical concerns.
* Objective: The doctor’s objective findings, including test results, measurements, or physical examination outcomes.
* Assessment: The doctor’s evaluation or diagnosis based on subjective and objective information.
* Plan: The proposed treatment plan, including medications, follow-ups, or other interventions.

While some encounter notes might include additional details, the majority conform to this semi-structured format, making them ideal for analysis and transformation.

* Goals for the Assignment

1. Transforming Encounter Notes:

Using an LLM to convert semi-structured encounter notes into a JSON format that organizes the information into structured fields. The JSON will include details such as demographics, encounter specifics, and the SOAP components of the note. Subsequently, you will need to transform the JSON data into a Parquet file, which is not only suitable for analysis in Spark but also ideal for storage later.
Here we will use the ML classificaition to assing the objective and assessment semi-structured fields into standardized, structured fields. The medical taxonomy for this task will be the one provided by the CDC, which defines standard codes for diagnoses, symptoms, procedures, and treatments. This step ensures the structured data aligns with domain-wide medical standards, making it interoperable and ready for deeper analysis.

The JSON format should capture the hierachies described in the structure below. 




2. Basic Analytics and Visualizations:
Using Apache Spark, perform comprehensive data analysis on the encounter data and create visualizations that reveal meaningful patterns. Your analysis must include:
- COVID-19 Case Demographics: Case breakdown by age ranges ([0-5], [6-10], [11-17], [18-30], [31-50], [51-70], [71+])
- Cumulative case count of Covid between the earliest case observed in the dataset and last case observed
- Symptoms for all COVID-19 patients versus patients that admitted into the intensive care unit due to COVID.
- Rank medications by frequency of prescription
- Analyze medication patterns across different demographic groups (e.g., top 3 per age group)
- Identify and plot co-morbidity information from the patient records (e.g., hypertension, obesity, prediabetes, etc.) provided in the dataset. 
- An independent group analysis: You need to develop and execute THREE original analyses that provide meaningful insights about COVID-19 patterns in this dataset. For each analysis:
  - Clearly state your analytical question/hypothesis
  - Justify why this analysis is valuable
  - Show your Spark code and methodology
  - Present results with appropriate visualizations


In [0]:
EncounterType:
    code
    description

Encounter:
    id
    date
    time
    type: EncounterType
    provider_id
    facility_id

Address:
    city
    state

Demographics:
    id
    name
    date_of_birth
    age
    gender
    address: Address
    insurance

Condition:
    code
    description

Medication:
    code
    description

Immunization:
    code
    description
    date: date

VitalMeasurement:
    code
    value: float
    unit

BloodPressure:
    systolic: VitalMeasurement
    diastolic: VitalMeasurement

CurrentVitals:
    temperature: VitalMeasurement
    heart_rate: VitalMeasurement
    blood_pressure: BloodPressure
    respiratory_rate: VitalMeasurement
    oxygen_saturation: VitalMeasurement
    weight: VitalMeasurement

BaselineVitals:
    date: date
    height: VitalMeasurement
    weight: VitalMeasurement
    bmi: VitalMeasurement
    bmi_percentile: VitalMeasurement

Vitals:
    current: CurrentVitals
    baseline: BaselineVitals

RespiratoryTest:
    code
    result

RespiratoryPanel:
    influenza_a: RespiratoryTest
    influenza_b: RespiratoryTest
    rsv: RespiratoryTest
    parainfluenza_1: RespiratoryTest
    parainfluenza_2: RespiratoryTest
    parainfluenza_3: RespiratoryTest
    rhinovirus: RespiratoryTest
    metapneumovirus: RespiratoryTest
    adenovirus: RespiratoryTest

Covid19Test:
    code
    description
    result

Laboratory:
    covid19: Covid19Test
    respiratory_panel: RespiratoryPanel

Procedure:
    code
    description
    date: date
    reasonCode
    reasonDescription

CarePlan:
    id
    code
    description
    start: date
    stop: date
    reasonCode
    reasonDescription

PatientRecord:
    encounter: Encounter
    demographics: Demographics
    conditions: List[Condition]
    medications: List[Medication]
    immunizations: List[Immunization]
    vitals: Vitals
    laboratory: Laboratory
    procedures: List[Procedure]


In [0]:
!pip install -U sentence-transformers
!pip install faiss-cpu
!pip install PyMuPDF Pillow numpy
!pip install --upgrade typing_extensions
!pip install sentence-transformers
dbutils.library.restartPython()

     |████████████████████████████████| 268 kB 7.1 MB/s 
     |████████████████████████████████| 78 kB 12.5 MB/s 
     |████████████████████████████████| 450 kB 82.6 MB/s 
     |████████▌                       | 2.7 MB 49.1 MB/s eta

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 16.8 MB 6.2 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
faiss-cpu 1.9.0.post1 requires numpy<3.0,>=1.25.0, but you have numpy 1.22.4 which is incompatible.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-f0811379-8e39-4182-a616-5912adc62c70/bin/python -m pip install --upgrade pip' command.


1. Transforming Encounter Notes:

Using an LLM to convert semi-structured encounter notes into a JSON format that organizes the information into structured fields. The JSON will include details such as demographics, encounter specifics, and the SOAP components of the note. Subsequently, you will need to transform the JSON data into a Parquet file, which is not only suitable for analysis in Spark but also ideal for storage later. Here we will use the ML classificaition to assing the objective and assessment semi-structured fields into standardized, structured fields. The medical taxonomy for this task will be the one provided by the CDC, which defines standard codes for diagnoses, symptoms, procedures, and treatments. This step ensures the structured data aligns with domain-wide medical standards, making it interoperable and ready for deeper analysis.

In [0]:
### NOT DONE YET


import re
import json
import pandas as pd
from pyspark.sql import SparkSession
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Initialize Spark session
spark = SparkSession.builder.appName("EncounterProcessing").getOrCreate()

# List of file paths
file_paths = [
    "/FileStore/tables/055ae6fc_7e18_4a39_8058_64082ca6d515.txt",
    "/FileStore/tables/199c586f_af16_4091_9998_ee4cfc02ee7a.txt",
    "/FileStore/tables/199c586f_af16_4091_9998_ee4cfc02ee7a_txt_",
    "/FileStore/tables/353016ea_a0ff_4154_85bb_1cf8b6cedf20.txt",
    "/FileStore/tables/28658715_b770_4576_9a81_fbb2282a98ea.txt",
    "/FileStore/tables/ae9efba3_ddc4_43f9_a781_f72019388548.txt",
    "/FileStore/tables/b9fd2dd8_181b_494b_ab15_e9f286d668d9.txt",
    "/FileStore/tables/df6b563d_1ff4_4833_9af8_84431e641e9c.txt",
    "/FileStore/tables/d22592ac_552f_4ecd_a63d_7663d77ce9ba.txt",
    "/FileStore/tables/f0f3bc8d_ef38_49ce_a2bd_dfdda982b271.txt",
    "/FileStore/tables/f73d6f41_0091_4485_8b43_9d38eb98fb36.txt"
]

# Helper functions to parse specific hierarchical sections
def parse_address(demographics):
    """Parse address from demographics text."""
    address_match = re.search(r"Address: (.+)", demographics)
    if address_match:
        address_parts = address_match.group(1).split(", ")
        return {
            "city": address_parts[0].strip() if len(address_parts) > 1 else None,
            "state": address_parts[1].strip() if len(address_parts) > 1 else None,
        }
    return None

def parse_vital_measurements(objective_section):
    """Parse vital measurements from Objective section."""
    temperature_match = re.search(r"Temperature: (.+)", objective_section)
    heart_rate_match = re.search(r"Heart Rate: (.+)", objective_section)
    respiratory_rate_match = re.search(r"Respiratory Rate: (.+)", objective_section)
    oxygen_saturation_match = re.search(r"O2 Saturation: (.+)", objective_section)
    weight_match = re.search(r"Weight: (.+)", objective_section)

    vitals = {
        "temperature": temperature_match.group(1) if temperature_match else None,
        "heart_rate": heart_rate_match.group(1) if heart_rate_match else None,
        "blood_pressure": {"systolic": None, "diastolic": None},
        "respiratory_rate": respiratory_rate_match.group(1) if respiratory_rate_match else None,
        "oxygen_saturation": oxygen_saturation_match.group(1) if oxygen_saturation_match else None,
        "weight": weight_match.group(1) if weight_match else None,
    }

    for key in vitals.keys():
        if key == "blood_pressure":
            bp_match = re.search(r"Blood Pressure: (\d+)/(\d+)", objective_section)
            if bp_match:
                vitals["blood_pressure"]["systolic"] = {"value": float(bp_match.group(1)), "unit": "mmHg"}
                vitals["blood_pressure"]["diastolic"] = {"value": float(bp_match.group(2)), "unit": "mmHg"}
    return vitals


def parse_vital_measurements2(objective_section):
    """Parse vital measurements from Objective section."""
    encounter_date_match = re.search(r"Date of Service: (.+?)\n", note)
    weight_match = re.search(r"Weight: (.+)", objective_section)
    height_match = re.search(r"Height: (.+)", objective_section)
    bmi_match = re.search(r"BMI: (.+)", objective_section)

    # Initialize index to None in case "percentile" isn't found in BMI
    index = None
    
    # Check if BMI contains "percentile" and find the index if applicable
    if bmi_match and bmi_match.group(1):
        search_index = [m.start() for m in re.finditer(r"percentile", bmi_match.group(1))]
        if len(bmi_match.group(1)) > 7 and search_index:
            index = search_index[0]
    
    # Extract vital measurements from the matches
    vitals = {
        "date": encounter_date_match.group(1) if encounter_date_match else None,
        "height": height_match.group(1) if height_match else None,
        "weight": weight_match.group(1) if weight_match else None,
        "bmi": bmi_match.group(1)[index - 19: index - 16] if bmi_match and len(bmi_match.group(1)) > 7 and index else bmi_match.group(1)[0:4] if bmi_match else None,
        "bmi_percentile": bmi_match.group(1)[index - 7: index - 1] if bmi_match and len(bmi_match.group(1)) > 7 and index else None,
    }
    
    return vitals


# Main parsing function
def parse_encounters(note, ename):
    """Parse a single encounter note."""
    encounter = {
        "encounter": {}, #
        "demographics": {}, #
        "conditions": [], 
        "medications": [],
        "immunizations": [],
        "vitals": {"current": {}, "baseline": {}}, #
        "laboratory": {"covid19": {}, "respiratory_panel": {}},
        "procedures": [],
    }
    
    # Extract encounter information
    encounter_id_match = ename[18:53]
    encounter_date_match = re.search(r"Date of Service: (.+?)\n", note)
    provider_id_match = re.search(r"Provider: (.+?)(\n|$)", note)
    facility_id_match = re.search(r"Facility: (.+?)(\n|$)", note)
    encounter["encounter"] = {
        "id": encounter_id_match if encounter_id_match else None,
        "date": encounter_date_match.group(1) if encounter_date_match else None,
        "type": {"code": "encounter_code_placeholder", "description": "encounter_type_placeholder"},
        "provider_id": provider_id_match.group(1) if provider_id_match else None,
        "facility_id": facility_id_match.group(1) if facility_id_match else None,
    }
    
    # Extract demographics
    demographics_match = re.search(r"DEMOGRAPHICS:\n(.+?)\n\n", note, re.DOTALL)
    if demographics_match:
        demographics = demographics_match.group(1)
        dob = re.search(r"DOB: (.+)", demographics).group(1)
        index = [m.start() for m in re.finditer(r"/", dob)][1]
        #print(re.search(r"DOB: (.+)", demographics).group(1)[index + 1: index + 5])
        encounter["demographics"] = {
            "id": ename[18:53],
            "name": re.search(r"Name: (.+)", demographics).group(1),
            "date_of_birth": re.search(r"DOB: (.+)", demographics).group(1),
            "age": 2024 - int(re.search(r"DOB: (.+)", demographics).group(1)[index + 1: index + 5]),
            "gender": re.search(r"Gender: (.+)", demographics).group(1),
            "address": parse_address(demographics),
            "insurance": re.search(r"Insurance: (.+)", demographics).group(1),
        }
    
    # Extract Objective section and vitals
    objective_match = re.search(r"OBJECTIVE:\n(.+?)(\n\n|$)", note, re.DOTALL)
    if objective_match:
        objective_section = objective_match.group(1)
        encounter["vitals"]["current"] = parse_vital_measurements(objective_section)
        encounter["vitals"]["baseline"] = parse_vital_measurements2(note)
    
    # Extract laboratory information           RNA DNA
    covid19
    influenza_a_match = re.search(r"Influenza A: (.+?)\n", note)
    influenza_a_match = re.search(r"Influenza A: (.+?)\n", note)
    influenza_b_match = re.search(r"Influenza B: (.+?)\n", note)
    rsv_match = re.search(r"RSV: (.+?)\n", note)
    parainfluenza_1_match = re.search(r"Parainfluenza 1: (.+?)\n", note)
    parainfluenza_2_match = re.search(r"Parainfluenza 2: (.+?)\n", note)
    parainfluenza_3_match = re.search(r"Parainfluenza 3: (.+?)\n", note)
    rhinovirus_match = re.search(r"Rhinovirus: (.+?)\n", note)
    metapneumovirus_match = re.search(r"Metapneumovirus: (.+?)\n", note)
    adenovirus_match = re.search(r"Adenovirus: (.+?)\n", note)
    encounter["laboratory"]["respiratory_panel"] = {
        "influenza_a": {"code": "resp_code_placeholder", "result":  influenza_a_match.group(1) if influenza_a_match else None,},
        "influenza_b": {"code": "resp_code_placeholder", "result":  influenza_b_match.group(1) if influenza_b_match else None,},
        "rsv": {"code": "resp_code_placeholder", "result":  rsv_match.group(1) if rsv_match else None,},
        "parainfluenza_1": {"code": "resp_code_placeholder", "result":  parainfluenza_1_match.group(1) if parainfluenza_1_match else None,},
        "parainfluenza_2": {"code": "resp_code_placeholder", "result":  parainfluenza_2_match.group(1) if parainfluenza_2_match else None,},
        "parainfluenza_3": {"code": "resp_code_placeholder", "result":  parainfluenza_3_match.group(1) if parainfluenza_3_match else None,},
        "rhinovirus": {"code": "resp_code_placeholder", "result":  rhinovirus_match.group(1) if rhinovirus_match else None,},
        "metapneumovirus": {"code": "resp_code_placeholder", "result":  metapneumovirus_match.group(1) if metapneumovirus_match else None,},
        "adenovirus": {"code": "resp_code_placeholder", "result":  adenovirus_match.group(1) if adenovirus_match else None,},
    }

    # Extract SOAP notes (conditions, assessments, and plans)
    encounter["soap"] = {}
    for section in ["SUBJECTIVE", "OBJECTIVE", "ASSESSMENT", "PLAN"]:
        match = re.search(fr"{section}:\n(.+?)(\n\n|$)", note, re.DOTALL)
        encounter["soap"][section.lower()] = match.group(1).strip() if match else None
    
    

    # Load pre-trained model
    model = SentenceTransformer('all-MiniLM-L6-v2')

    # Example Objective and Assessment data
    objective_text = encounter["soap"]["objective"]
    assessment_text = encounter["soap"]["assessment"]

    # Generate embeddings
    objective_embeddings = model.encode(objective_text)
    assessment_embeddings = model.encode(assessment_text)

    #print("Objective Embedding Shape:", np.array(objective_embeddings).shape)
    #print("Assessment Embedding Shape:", np.array(assessment_embeddings).shape)



    return encounter

# Process each file
parsed_data = []
for file_path in file_paths:
    # Read file content into a string
    file_df = spark.read.text(file_path)
    note = "\n".join(row["value"] for row in file_df.collect())  # Combine lines into a single string
    parsed_data.append(parse_encounters(note, file_path))

# Save parsed data as JSON
with open("parsed_encounters.json", "w") as json_file:
    json.dump(parsed_data, json_file, indent=4)

# Convert parsed data to Pandas DataFrame
parsed_df = pd.json_normalize(parsed_data)

# Save to Parquet
parsed_df.to_parquet("encounter_notes.parquet", engine="pyarrow", index=False)

# Display the DataFrame (optional)
display(parsed_df)


conditions,medications,immunizations,procedures,encounter.id,encounter.date,encounter.type.code,encounter.type.description,encounter.provider_id,encounter.facility_id,demographics.id,demographics.name,demographics.date_of_birth,demographics.age,demographics.gender,demographics.address.city,demographics.address.state,demographics.insurance,vitals.current.temperature,vitals.current.heart_rate,vitals.current.blood_pressure.systolic.value,vitals.current.blood_pressure.systolic.unit,vitals.current.blood_pressure.diastolic.value,vitals.current.blood_pressure.diastolic.unit,vitals.current.respiratory_rate,vitals.current.oxygen_saturation,vitals.current.weight,vitals.baseline.date,vitals.baseline.height,vitals.baseline.weight,vitals.baseline.bmi,vitals.baseline.bmi_percentile,laboratory.respiratory_panel.influenza_a.code,laboratory.respiratory_panel.influenza_a.result,laboratory.respiratory_panel.influenza_b.code,laboratory.respiratory_panel.influenza_b.result,laboratory.respiratory_panel.rsv.code,laboratory.respiratory_panel.rsv.result,laboratory.respiratory_panel.parainfluenza_1.code,laboratory.respiratory_panel.parainfluenza_1.result,laboratory.respiratory_panel.parainfluenza_2.code,laboratory.respiratory_panel.parainfluenza_2.result,laboratory.respiratory_panel.parainfluenza_3.code,laboratory.respiratory_panel.parainfluenza_3.result,laboratory.respiratory_panel.rhinovirus.code,laboratory.respiratory_panel.rhinovirus.result,laboratory.respiratory_panel.metapneumovirus.code,laboratory.respiratory_panel.metapneumovirus.result,laboratory.respiratory_panel.adenovirus.code,laboratory.respiratory_panel.adenovirus.result,soap.subjective,soap.objective,soap.assessment,soap.plan
List(),List(),List(),List(),055ae6fc_7e18_4a39_8058_64082ca6d51,"March 2, 2020 15:45-16:30",encounter_code_placeholder,encounter_type_placeholder,ID# e2c226c2-3e1e-3d0b-b997-ce9544c10528,5103c940-0c08-392f-95cd-446e0cea042a,055ae6fc_7e18_4a39_8058_64082ca6d51,Jeffrey Greenfelder,1/16/2005,19,Male,428 Wiza Glen Unit 91,Springfield,Guardian,39.3°C (102.7°F),131.1/min,120.0,mmHg,73.0,mmHg,27.6/min,75.8% on room air,81.2 kg (stable),"March 2, 2020 15:45-16:30",155.0 cm,81.2 kg,33.,99.1th,resp_code_placeholder,null,resp_code_placeholder,null,resp_code_placeholder,null,resp_code_placeholder,null,resp_code_placeholder,null,resp_code_placeholder,null,resp_code_placeholder,null,resp_code_placeholder,null,resp_code_placeholder,null,"Adolescent patient presents with multiple symptoms, including cough, sore throat, severe fatigue, muscle pain, joint pain, and fever. The patient has no history of smoking and reports that the symptoms began recently.",Vitals: Temperature: 39.3°C (102.7°F) Heart Rate: 131.1/min Blood Pressure: 120/73 mmHg Respiratory Rate: 27.6/min O2 Saturation: 75.8% on room air Weight: 81.2 kg (stable),1. Suspected COVID-19 with severe symptoms 2. Severe hypoxemia requiring immediate intervention 3. Tachycardia (HR 131) 4. High-grade fever 5. Risk factors: - Obesity (BMI 33.8) - Adolescent age,1. Face mask provided for immediate oxygen support 2. Infectious disease care plan initiated 3. Close monitoring required due to: - Severe hypoxemia - Tachycardia - Age and obesity risk factors 4. Parent/patient education on: - Home isolation protocols - Warning signs requiring emergency care - Return precautions 5. Follow-up plan: - Daily monitoring during acute phase - Virtual check-ins as needed
List(),List(),List(),List(),199c586f_af16_4091_9998_ee4cfc02ee7,"March 2, 2020 04:15-05:15",encounter_code_placeholder,encounter_type_placeholder,ID# 8be741d6-44d0-3412-88dd-4987e1559f0b,b1ddf812-1fdd-3adf-b1d5-32cc8bd07ebb,199c586f_af16_4091_9998_ee4cfc02ee7,Jimmie Harris,1/9/2004 (16y/o),20,Female,Pembroke,MA,Medicare/Medicaid,40.7°C (105.3°F),98/min,120.0,mmHg,89.0,mmHg,22/min,78.2% on room air,45.3 kg (stable),"March 2, 2020 04:15-05:15",149.3 cm,45.3 kg (stable),20.,47.6th,resp_code_placeholder,null,resp_code_placeholder,null,resp_code_placeholder,Negative,resp_code_placeh